In [1]:
import yaml
import torch
from omegaconf import OmegaConf
from taming.models.vqgan import VQModel, GumbelVQ
from PIL import Image
import PIL
import numpy as np
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import cv2
import onnx

def load_config(config_path, display=False):
  config = OmegaConf.load(config_path)
  if display:
    print(yaml.dump(OmegaConf.to_container(config)))
  return config

def load_vqgan(config, ckpt_path=None, is_gumbel=False):
  if is_gumbel:
    model = GumbelVQ(**config.model.params)
  else:
    model = VQModel(**config.model.params)
  if ckpt_path is not None:
    sd = torch.load(ckpt_path, map_location="cuda")["state_dict"]
    missing, unexpected = model.load_state_dict(sd, strict=False)
  return model

In [2]:
config1024 = load_config("configs/model.yaml", display=False)
#model1024 = load_vqgan(config1024, ckpt_path="ckpts/last.ckpt").to('cuda')
#model1024 = load_vqgan(config1024, ckpt_path="ckpts/last_full_train.ckpt").to('cuda')
model1024 = load_vqgan(config1024, ckpt_path="configs/last.ckpt")

Working with z of shape (1, 256, 16, 16) = 65536 dimensions.


e:\taming\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
e:\taming\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


loaded pretrained LPIPS loss from taming/modules/autoencoder/lpips\vgg.pth
VQLPIPSWithDiscriminator running with hinge loss.


In [3]:
torch_input = torch.randn(1, 3, 160, 256)
onnx_program = torch.onnx.dynamo_export(model1024, torch_input)

e:\taming\lib\site-packages\torch\onnx\_internal\exporter.py:136: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [4]:
onnx_program.save("vqgan.onnx")

In [6]:
torch.onnx.export(model1024, torch_input, "rtt.onnx")

e:\taming\lib\site-packages\torch\onnx\utils.py:702: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_graph_shape_type_inference(
e:\taming\lib\site-packages\torch\onnx\utils.py:1208: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_graph_shape_type_inference(


In [25]:
p = model1024.encoder

In [26]:
torch.onnx.export(p, torch_input, "rencode.onnx")

e:\taming-transformers\taming\modules\diffusionmodules\model.py:181: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  w_ = w_ * (int(c)**(-0.5))
e:\taming\lib\site-packages\torch\onnx\_internal\jit_utils.py:307: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
e:\taming\lib\site-packages\torch\onnx\utils.py:702: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc

In [9]:
enc = onnx.load('rencode.onnx')

In [10]:
onnx.checker.check_model(enc)

In [22]:
import torch.nn as nn
class EncoderManual(nn.Module):
    def __init__(self, model1024):
        super(EncoderManual, self).__init__()
        # Assuming encoder layers are defined in a sequential manner in original_model.encoder
        self.encoder = model1024.encoder
        self.quant_conv = model1024.quant_conv
        self.quantize = model1024.quantize

    def forward(self, x):
        h = self.encoder(x)
        h = self.quant_conv(h)
        quant, emb_loss, info = self.quantize(h)
        return [quant, emb_loss, info]


In [23]:
a=EncoderManual(model1024)

In [24]:
torch.onnx.export(a, torch_input, 'quant3.onnx', input_names=['input'])

e:\taming-transformers\taming\modules\diffusionmodules\model.py:181: TracerWarning: Converting a tensor to a Python integer might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  w_ = w_ * (int(c)**(-0.5))
e:\taming\lib\site-packages\torch\onnx\_internal\jit_utils.py:307: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\jit\passes\onnx\constant_fold.cpp:181.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
e:\taming\lib\site-packages\torch\onnx\utils.py:702: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc